# EFM Clustering Scale and H₀ Simulation (S/T State, N=500)

This notebook simulates the Ehokolo Fluxon Model (EFM) in the Space/Time (S/T) state to derive the clustering scale and Hubble constant (H₀) from first principles. We use a 500³ grid (L=1000 Mpc) to capture large-scale structure formation, targeting a clustering scale of ~628 Mpc (DESI/SDSS) and H₀ consistent with SHOES (73.0 ± 1.0 km/s/Mpc) and Planck (67.4 ± 0.5 km/s/Mpc).

## Objectives
- Simulate cosmological structure formation using the EFM NLKG equation in the S/T state.
- Use a properly normalized Gaussian random field initial condition with a Harrison-Zel'dovich power spectrum.
- Compute the clustering scale via the correlation function and H₀ via velocity field analysis.
- Optimize for Google Colab Pro+ (A100 GPU, 40 GB VRAM, 80 GB RAM) to scale up to 1000³.
- Validate results against DESI/SDSS, SHOES, and Planck datasets.

## Hardware
- GPU: NVIDIA A100 (40 GB VRAM)
- System RAM: 80 GB
- Environment: Google Colab Pro+

## Setup Instructions
1. Set runtime to A100 GPU.
2. Execute all cells sequentially.
3. Monitor VRAM (<20 GB) and RAM (<40 GB) usage.
4. Save outputs to Google Drive.

## 1. Setup: Libraries, GPU Check, Drive Mount, Paths

In [ ]:
# Clear GPU memory
import torch
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

# Install/Import libraries
!nvidia-smi
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import psutil
import time
from datetime import datetime
from google.colab import drive
import os
import scipy.fft as fft

# Check GPU and memory
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Mount Google Drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/EFM_Simulations/'
sim_path = os.path.join(base_path, 'Clustering_Scale_Simulation/')
checkpoint_path = os.path.join(sim_path, 'checkpoints/')
data_path = os.path.join(sim_path, 'data/')
plot_path = os.path.join(sim_path, 'plots/')
os.makedirs(checkpoint_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)
print(f"Paths created/checked:\n Checkpoints: {checkpoint_path}\n Data: {data_path}\n Plots: {plot_path}")

## 2. Simulation Parameters (S/T State, N=500)

In [ ]:
# Numerical Parameters
N = 500  # Grid size
L = 1000.0  # Box size (Mpc)
dx = L / N  # Spatial step (Mpc)
dx_m = dx * 3.086e22  # Spatial step (meters)
dt = 1e11  # Time step (seconds, ~3,170 years)
T_steps = 1000  # Total steps (~3.17e7 years)
save_interval = 100
checkpoint_interval = 500

# Physical Parameters
c = 3e8  # Speed of light (m/s)
m = 0.0  # Massless field for cosmology
g = 0.01  # Reduced cubic nonlinearity
eta = 0.001  # Reduced quintic nonlinearity
alpha = 0.1
delta = 0.05
gamma_damp = 0.1  # Linear damping
k = 0.01  # Density coupling
G = 6.674e-11  # Gravitational constant
rho_ref = 1.5

# Initial Conditions
A = 1e-5  # Amplitude of primordial fluctuations
ns = 0.96  # Spectral index (Harrison-Zel'dovich)

# Precision
dtype = torch.float16

print("--- S/T Clustering Simulation Parameters ---")
print(f"Grid Size (N): {N}^3")
print(f"Box Size (L): {L} Mpc")
print(f"Time Step (dt): {dt:.2e} seconds")
print(f"Total Steps: {T_steps}")
print(f"Precision: {dtype}")

## 3. Helper Functions

In [ ]:
# Potential function V(phi) = m*phi^2 + g*phi^4 + eta*phi^6
def potential(phi, m_p, g_p, eta_p):
    phi_f32 = phi.to(torch.float32)
    m_p_f32 = torch.tensor(m_p, dtype=torch.float32, device=phi.device)
    g_p_f32 = torch.tensor(g_p, dtype=torch.float32, device=phi.device)
    eta_p_f32 = torch.tensor(eta_p, dtype=torch.float32, device=phi.device)
    term2 = m_p_f32 * phi_f32**2
    term4 = g_p_f32 * phi_f32**4
    term6 = eta_p_f32 * phi_f32**6
    return (term2 + term4 + term6).to(phi.dtype)

# NLKG derivative with linear damping
def nlkg_derivative(phi, phi_dot, m_p, g_p, eta_p, c_p, alpha_p, delta_p, gamma_damp_p, k_p, G_p, dx_p, N_p, device_p):
    phi_f32 = phi.to(torch.float32)
    phi_dot_f32 = phi_dot.to(torch.float32)
    dx_p_f32 = torch.tensor(dx_p, dtype=torch.float32, device=device_p)
    
    # Laplacian
    laplacian_f32 = torch.zeros_like(phi_f32)
    for dim in range(3):
        laplacian_f32 += torch.roll(phi_f32, shifts=1, dims=dim)
        laplacian_f32 += torch.roll(phi_f32, shifts=-1, dims=dim)
    laplacian = (laplacian_f32 - 6.0 * phi_f32) / dx_p_f32**2

    # V'(phi)
    dV_dphi = 2 * m_p * phi_f32 + 4 * g_p * phi_f32**3 + 6 * eta_p * phi_f32**5
    
    # Coupling and dissipation terms
    grad_phi = torch.stack([torch.gradient(phi_f32, spacing=dx_p_f32, dim=d)[0] for d in range(3)])
    alpha_term = alpha_p * phi_f32 * (phi_dot_f32 * grad_phi[0])
    dissipation = delta_p * (phi_dot_f32**2) * phi_f32
    linear_damping = -gamma_damp_p * phi_dot_f32
    gravity = 8 * np.pi * G_p * k_p * phi_f32**2
    
    # phi_ddot
    c_p_f32 = torch.tensor(c_p, dtype=torch.float32, device=device_p)
    phi_ddot = c_p_f32**2 * laplacian - dV_dphi + alpha_term + dissipation + linear_damping - gravity
    
    # Absorbing boundary conditions
    boundary_width = int(0.1 * N_p)
    damping_factor = 0.05
    mask = torch.ones_like(phi)
    indices = torch.arange(N_p, device=device_p, dtype=phi.dtype)
    damping_profile = torch.ones(N_p, device=device_p, dtype=phi.dtype)
    ramp = torch.linspace(1.0, damping_factor, boundary_width, device=device_p, dtype=phi.dtype)
    damping_profile[:boundary_width] = ramp.flip(dims=[0])
    damping_profile[-boundary_width:] = ramp
    for dim in range(3):
        if dim == 0: mask *= damping_profile[:, None, None]
        elif dim == 1: mask *= damping_profile[None, :, None]
        else: mask *= damping_profile[None, None, :]
    
    phi_dot_damped = phi_dot_f32 * mask.to(torch.float32)
    phi_ddot_damped = phi_ddot * mask.to(torch.float32)
    return phi_dot_damped.to(phi_dot.dtype), phi_ddot_damped.to(phi_ddot.dtype)

# RK4 integrator
def update_phi_rk4(phi, phi_dot, dt_p, m_p, g_p, eta_p, c_p, alpha_p, delta_p, gamma_damp_p, k_p, G_p, dx_p, N_p, device_p):
    with torch.no_grad():
        dt_p_f32 = torch.tensor(dt_p, dtype=torch.float32, device=device_p)
        k1_v, k1_a = nlkg_derivative(phi, phi_dot, m_p, g_p, eta_p, c_p, alpha_p, delta_p, gamma_damp_p, k_p, G_p, dx_p, N_p, device_p)
        k2_v, k2_a = nlkg_derivative(phi + 0.5 * dt_p_f32 * k1_v, phi_dot + 0.5 * dt_p_f32 * k1_a, m_p, g_p, eta_p, c_p, alpha_p, delta_p, gamma_damp_p, k_p, G_p, dx_p, N_p, device_p)
        k3_v, k3_a = nlkg_derivative(phi + 0.5 * dt_p_f32 * k2_v, phi_dot + 0.5 * dt_p_f32 * k2_a, m_p, g_p, eta_p, c_p, alpha_p, delta_p, gamma_damp_p, k_p, G_p, dx_p, N_p, device_p)
        k4_v, k4_a = nlkg_derivative(phi + dt_p_f32 * k3_v, phi_dot + dt_p_f32 * k3_a, m_p, g_p, eta_p, c_p, alpha_p, delta_p, gamma_damp_p, k_p, G_p, dx_p, N_p, device_p)
        phi_new = phi + (dt_p_f32 / 6.0) * (k1_v + 2.0 * k2_v + 2.0 * k3_v + k4_v)
        phi_dot_new = phi_dot + (dt_p_f32 / 6.0) * (k1_a + 2.0 * k2_a + 2.0 * k3_a + k4_a)
        phi_new.clamp_(-1e-3, 1e-3)  # Clamp to prevent runaway growth
        phi_dot_new.clamp_(-1e-3, 1e-3)
        return phi_new, phi_dot_new

# Metrics computation
def compute_metrics(phi, phi_dot, k_p, m_p, g_p, eta_p, c_p, dx_p, device_p):
    with torch.no_grad():
        phi_f32 = phi.to(torch.float32)
        phi_dot_f32 = phi_dot.to(torch.float32)
        dx_p_f32 = torch.tensor(dx_p, dtype=torch.float32, device=device_p)
        c_p_f32 = torch.tensor(c_p, dtype=torch.float32, device=device_p)

        max_amp = torch.max(torch.abs(phi_f32)).item()
        avg_density = k_p * torch.mean(phi_f32**2).item()

        kinetic = 0.5 * phi_dot_f32**2
        grad_phi = torch.stack([torch.gradient(phi_f32, spacing=dx_p_f32, dim=d)[0] for d in range(3)])
        gradient = 0.5 * c_p_f32**2 * (grad_phi[0]**2 + grad_phi[1]**2 + grad_phi[2]**2)
        potential_energy = potential(phi_f32, m_p, g_p, eta_p)
        total_energy = (kinetic + gradient + potential_energy).sum().item() * (dx_p_f32.item()**3)

        if not (np.isfinite(max_amp) and np.isfinite(avg_density) and np.isfinite(total_energy)):
            return float('nan'), float('nan'), float('nan')
        return max_amp, avg_density, total_energy

# Initial conditions: Properly normalized Gaussian random field
def generate_initial_conditions(N, L, A, ns):
    kx = fft.fftfreq(N, d=dx)
    ky = fft.fftfreq(N, d=dx)
    kz = fft.fftfreq(N, d=dx)
    KX, KY, KZ = np.meshgrid(kx, ky, kz, indexing='ij')
    k = np.sqrt(KX**2 + KY**2 + KZ**2)
    k[k == 0] = 1e-10
    Pk = A * (k ** ns)
    delta_k = np.random.normal(0, np.sqrt(Pk/2), (N, N, N)) + 1j * np.random.normal(0, np.sqrt(Pk/2), (N, N, N))
    delta = fft.ifftn(delta_k) * (N**3)
    delta = np.real(delta)
    # Normalize to RMS ~ A
    rms = np.sqrt(np.mean(delta**2))
    if rms > 0:
        delta = delta * (A / rms)
    return delta

## 4. Simulation Execution

In [ ]:
print(f"\n--- Starting S/T State Simulation ---")
sim_start_time = time.time()

# Initialize fields
init_path = f"{data_path}initial_conditions_N{N}_L{L}.npz"
if not os.path.exists(init_path):
    phi_ST = generate_initial_conditions(N, L, A, ns)
    phi_dot_ST = np.zeros((N, N, N))
    np.savez_compressed(init_path, phi_ST=phi_ST, phi_dot_ST=phi_dot_ST)
    print("Initial conditions saved.")
else:
    init_data = np.load(init_path)
    phi_ST = init_data['phi_ST']
    phi_dot_ST = init_data['phi_dot_ST']
    print("Initial conditions loaded.")

phi = torch.from_numpy(phi_ST).to(device, dtype=dtype)
phi_dot = torch.from_numpy(phi_dot_ST).to(device, dtype=dtype)

# Initial metrics
initial_max_amp, initial_avg_rho, initial_energy = compute_metrics(phi, phi_dot, k, m, g, eta, c, dx_m, device)
print(f"Initial Metrics: Max|φ|={initial_max_amp:.2e}, <ρ>={initial_avg_rho:.2e}, E={initial_energy:.2e}")

# Simulation loop
max_amp_history = [initial_max_amp]
avg_rho_history = [initial_avg_rho]
energy_history = [initial_energy]
stable = True
steps_completed = 0

pbar = tqdm(range(T_steps), desc="S/T State Simulation")
for t in pbar:
    try:
        phi, phi_dot = update_phi_rk4(phi, phi_dot, dt, m, g, eta, c, alpha, delta, gamma_damp, k, G, dx_m, N, device)
        steps_completed = t + 1
    except Exception as e:
        print(f"Error at step {t}: {e}")
        stable = False
        break

    if t % save_interval == 0 or t == T_steps - 1:
        max_amp, avg_density, total_energy = compute_metrics(phi, phi_dot, k, m, g, eta, c, dx_m, device)
        if not (np.isfinite(max_amp) and np.isfinite(avg_density) and np.isfinite(total_energy)) or max_amp > 1e-2:
            print(f"Instability at step {t}: Max|φ|={max_amp:.2e}, <ρ>={avg_density:.2e}, E={total_energy:.2e}")
            stable = False
            break
        max_amp_history.append(max_amp)
        avg_rho_history.append(avg_density)
        energy_history.append(total_energy)
        pbar.set_postfix({'Max|φ|': f'{max_amp:.2e}', '<ρ>': f'{avg_density:.2e}', 'E': f'{total_energy:.2e}'})

    if t % checkpoint_interval == 0 and t > 0:
        torch.save({'step': t, 'phi': phi.cpu(), 'phi_dot': phi_dot.cpu()}, os.path.join(checkpoint_path, f"step{t}.pt"))
        phi = phi.to(device)
        phi_dot = phi_dot.to(device)

pbar.close()

# Save final state
max_amp_history = np.array(max_amp_history, dtype=np.float32)
avg_rho_history = np.array(avg_rho_history, dtype=np.float32)
energy_history = np.array(energy_history, dtype=np.float32)
max_amp_history = np.where(np.isfinite(max_amp_history), max_amp_history, 0)
avg_rho_history = np.where(np.isfinite(avg_rho_history), avg_rho_history, 0)
energy_history = np.where(np.isfinite(energy_history), energy_history, 0)

final_results = {
    'stable': stable,
    'steps_completed': steps_completed,
    'max_amp': max_amp_history,
    'avg_rho': avg_rho_history,
    'energy': energy_history,
    'final_phi': phi.cpu().numpy() if stable else None,
    'final_phi_dot': phi_dot.cpu().numpy() if stable else None
}
np.savez(os.path.join(data_path, f"results_N{N}_T{T_steps}.npz"), **final_results)
print(f"Saved results to {data_path}")

sim_end_time = time.time()
print(f"Simulation completed in {(sim_end_time - sim_start_time) / 60:.2f} minutes")

## 5. Analysis: Clustering Scale and H₀

In [ ]:
# Load results
results_file = os.path.join(data_path, f"results_N{N}_T{T_steps}.npz")
results = np.load(results_file, allow_pickle=True)
results = {key: results[key].item() if results[key].ndim == 0 else results[key] for key in results.files}

if results['stable']:
    phi = torch.from_numpy(results['final_phi']).to(device, dtype=dtype)
    phi_dot = torch.from_numpy(results['final_phi_dot']).to(device, dtype=dtype)

    # Compute density and velocity
    rho = k * phi**2
    velocity = phi_dot
    positions = torch.linspace(-L/2, L/2, N, device=device)
    X, Y, Z = torch.meshgrid(positions, positions, positions, indexing='ij')
    r = torch.sqrt(X**2 + Y**2 + Z**2)
    v_r = velocity * (X/r + Y/r + Z/r)
    mask = (r > 0) & (r < 500)
    H0 = torch.mean(v_r[mask] / r[mask]).item() * 3.086e19  # km/s/Mpc

    # Correlation function
    def compute_correlation_function(phi):
        phi_slice = phi[N//2, :, :].cpu().numpy()
        phi_flat = phi_slice.flatten()
        corr = np.correlate(phi_flat, phi_flat, mode='full')
        r = np.linspace(-L/2, L/2, len(corr))
        return r, corr / np.max(corr)

    r, corr_func = compute_correlation_function(rho)
    r_peak = r[np.argmax(corr_func)]

    # Validation
    print(f"Clustering Scale: {r_peak:.2f} Mpc (DESI/SDSS: ~628 Mpc)")
    print(f"H0: {H0:.2f} km/s/Mpc (SHOES: 73.0 ± 1.0, Planck: 67.4 ± 0.5)")
    chi2_shoes = ((H0 - 73.0) / 1.0)**2
    chi2_planck = ((H0 - 67.4) / 0.5)**2
    print(f"χ² (SHOES): {chi2_shoes:.2f}, χ² (Planck): {chi2_planck:.2f}")

    # Plots
    plt.figure(figsize=(10, 5))
    plt.plot(r, corr_func, label='Correlation Function')
    plt.xlabel('r (Mpc)')
    plt.ylabel('Correlation')
    plt.title('Correlation Function')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(plot_path, f"correlation_N{N}_T{T_steps}.png"))
    plt.show()
else:
    print("Simulation was unstable, cannot compute observables.")